# Sorghum growth

In [1]:
from __future__ import annotations

import os
import sys
import time as t
from datetime import date
from multiprocessing import Pool

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

sys.path.append("../data") 
from archi_dict import archi_sorghum as archi
from oawidgets.plantgl import *  # noqa: F403

from openalea.archicrop.display import build_scene, display_scene  # noqa: F401
from openalea.archicrop.simulation import (
    plot_constained_vs_pot,
    plot_constrainted_vs_realized,
    plot_faPAR,
    plot_PAR,
    run_simulations,
    write_netcdf,
)
from openalea.plantgl.all import Color3, Material, Scene  # noqa: F401

%gui qt

In [2]:
stics_color = "orange"
archicrop_color = "green"

In [3]:
tec_file_xml='../data/02NT18SorgV2D1_tec.xml'
plt_file_xml='../data/sorgho_imp_M_v10_plt.xml'
stics_output_file='../data/mod_s02NT18SorgV2D1.sti'
weather_file = '../data/ntarla_corr.2018'
location = {  
'longitude': 3.87,
'latitude': 12.58,
'altitude': 800,
'timezone': 'Europe/Paris'}

In [4]:
seed = 18

In [5]:

start_time = t.time()

daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density = run_simulations(
    archi_params=archi, 
    tec_file=tec_file_xml, 
    plant_file=plt_file_xml, 
    dynamics_file=stics_output_file, 
    weather_file=weather_file,
    location=location,
    n_samples=10,
    latin_hypercube=True,
    opt_filter_organ_duration=False,
    opt_filter_pot_growth=True,
    opt_filter_realized_growth=True,
    light_inter=False,
    direct=False,
    error_LA_pot=1,
    error_height_pot=1,
    error_LA_realized=1,
    error_height_realized=1,
    seed=seed)

end_time = t.time()

elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.4f} minutes for {len(realized_la)} simulations")  # noqa: T201


[66.59621, 77.78567, 88.93402, 101.61795, 114.16251, 125.17745, 137.91595, 151.12643, 162.77878, 176.0939, 188.936, 203.01031, 216.95804, 230.77347, 245.69185, 259.4848, 270.37756, 283.1174, 294.06958, 303.95944, 315.65265, 328.14175, 339.7291, 352.11249, 365.84802, 380.16913, 390.92734, 403.0976, 413.73465, 426.44321, 437.43277, 449.33139, 460.66211, 472.50479, 484.23907, 497.33984, 508.28189, 520.50848, 533.42249, 544.55872, 556.75494, 570.22986, 582.92712, 596.07257, 608.77417, 621.21246, 634.22858, 647.85291, 661.76276, 676.47937, 689.44556, 701.35205, 714.61633, 727.85406, 740.82471, 754.60095, 769.96405, 783.55774, 794.35315, 807.88275, 819.21216, 833.02356, 846.51978, 858.12982, 872.45099, 886.05048, 900.71838, 914.80658, 929.35333, 943.39929, 959.28473, 972.47046, 985.5481, 999.98267, 1012.93463, 1028.5199, 1044.65417, 1055.44727, 1070.23096, 1082.39819, 1097.86218, 1115.65259, 1132.88574, 1151.25171, 1169.83484, 1184.73206, 1200.89099, 1216.04736, 1231.4574, 1249.33313, 1267.1

In [6]:
# Retrieve STICS growth and senescence dynamics
dates = [value["Date"] for value in daily_dynamics.values()]
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]
par_incident = [value["Incident PAR"] for value in daily_dynamics.values()]
par_stics = [value["Absorbed PAR"] for value in daily_dynamics.values()]

In [7]:
write_netcdf("sorghum_growth_test", daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density, seed)